In [2]:
import os
import re
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from transformers import GPT2Tokenizer, TFGPT2Model, GPT2Config

ImportError: cannot import name 'experimental_functions_run_eagerly' from 'tensorflow.python.eager.def_function' (c:\Users\badje\OneDrive\Bureau\Cours_M2\NLP\projet\Recipe_generator\venv\lib\site-packages\tensorflow\python\eager\def_function.py)

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', lang='fr')

In [8]:
current_dir = os.path.dirname(os.path.realpath("__file__"))
repo_dir = os.path.dirname(current_dir)
DATA_FILE = os.path.join(repo_dir,"data","cleaned_data.csv")
data = pd.read_csv(DATA_FILE, encoding="utf-8",sep=',')

In [17]:
data['Type'] = df["Type"].to_list()

In [13]:
data

,Nom,Ingredients,Ustensiles,Etapes,recettes
0,soupe veloute potimarron pomme terre,2 pomme terre 2 oignon hache 3 gousse ail ha...,louche mixeur plonger blender chauffer couteau...,etape 1 enlever ecorce pepin potimarron obliga...,soupe veloute potimarron pomme terre 2 pomme ...
1,soupe oignon,4 oignon 50 gramme beurre 1 c.a. huile 1...,louche mixeur plonger blender chauffer saladie...,etape 1 pelez emincer oignon etape 2 faire rev...,soupe oignon 4 oignon 50 gramme beurre 1 ...
2,veloute potiron,1 oignon 1 quartier potiron 1 brique creme f...,louche mixeur plonger couteau cocotte fonte po...,etape 1 couper chair potiron gros de etape 2 c...,veloute potiron 1 oignon 1 quartier potiron...
3,veloute butternut cannelle,1 oignon 1 c.a. creme frais 1 pincee canne...,louche mixeur plonger couteau cuillere bois po...,etape 1 couper butternut gros cube etape 2 cou...,veloute butternut cannelle 1 oignon 1 c.a. ...
4,tarte poireau lardon,250 gramme farine 140 gramme beurre 2 c....,moule tarte four couteau pinceau rape poele sa...,etape 1 prechauffer four 210 degre c thermosta...,tarte poireau lardon 250 gramme farine 14...
...,...,...,...,...,...
47335,liqueur genepi,40 morceau sucre 40 brin genepi 1 l alco...,entonnoir balance cuisine,etape 1 transvaser litre alcool gros bouteille...,liqueur genepi 40 morceau sucre 40 brin g...
47336,caipirinha,1 dose jus citron vert frais 15 millilitre j...,cuillere bois shaker,etape 1 shaker melanger ingredient glace pilee...,caipirinha 1 dose jus citron vert frais 15 ...
47337,kombucha maison citron gingembre,1 morceau gingembre raper 100 gramme sucre...,casserole balance cuisine,etape 1 commencer realiser sirop chauffer eau ...,kombucha maison citron gingembre 1 morceau ...
47338,gin tonic,4 cl gin 2 rondelle citron 8 cl tonic,NaN,etape 1 verser glaçon verre etape 2 ajouter gi...,gin tonic 4 cl gin 2 rondelle citron 8 ...


In [16]:
data['tokens'] = data['recettes'].apply(tokenizer.tokenize)

In [44]:
X = np.array(data['tokens'].to_list())
y = np.array(data['Type'].to_list())

oov_tokens = ['s','Entrée']
tokenizer.add_tokens(oov_tokens)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

max_length = 512
sequences = tokenizer.batch_encode_plus(data['recettes'].to_list(), padding=True, truncation=True, return_tensors='tf')

# num_classes = len(np.unique(y))
# y = tf.keras.utils.to_categorical(y, num_classes)

C:\Users\badje\AppData\Local\Temp\ipykernel_25620\1383507977.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.array(data['tokens'].to_list())


In [41]:
sequences['input_ids'].shape

TensorShape([47340, 1024])

In [46]:
# Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(sequences['input_ids'].numpy(), sequences['attention_mask'].numpy(), test_size=0.2, random_state=42)
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

X = sequences['input_ids'].numpy()
y = sequences['attention_mask'].numpy()
input_dim = X.shape[1]

# Reshape the input data to a 3D tensor
batch_size = X.shape[0]
sequence_length = 1  # Set the sequence length to 1
X = np.reshape(X, (batch_size, sequence_length, input_dim))

output_dim = y.shape[1]

# Reshape the target data to a 2D tensor
Y = np.reshape(y, (batch_size, output_dim))

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [59]:
X_train

array([[[   83, 10602,   502, ..., 50257, 50257, 50257]],

       [[37424,    68,  1093, ..., 50257, 50257, 50257]],

       [[   70,   559, 19503, ..., 50257, 50257, 50257]],

       ...,

       [[   79,   378, 17463, ..., 50257, 50257, 50257]],

       [[  462, 21348,  3450, ..., 50257, 50257, 50257]],

       [[   66, 14501, 48241, ..., 50257, 50257, 50257]]])

In [5]:
model = TFGPT2Model.from_pretrained('gpt2')
config = GPT2Config.from_pretrained('gpt2')
config.output_hidden_states = True
model.config = config

Downloading: 100%|██████████| 498M/498M [06:47<00:00, 1.22MB/s] 
c:\Users\badje\OneDrive\Bureau\Cours_M2\NLP\projet\Recipe_generator\venv\lib\site-packages\huggingface_hub\file_download.py:127: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\badje\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
All model checkpoint layers were use

In [58]:
# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.CategoricalAccuracy('accuracy')

model.compile(
  loss=loss,
  optimizer=optimizer,
  metrics=metric
)

# Train the model
history = model.fit(
  X_train,
  y_train,
  # batch_size=batch_size,
  epochs=10,
  validation_data=(X_test, y_test)
)


Epoch 1/10


AttributeError: in user code:

    File "c:\Users\badje\OneDrive\Bureau\Cours_M2\NLP\projet\Recipe_generator\venv\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\badje\OneDrive\Bureau\Cours_M2\NLP\projet\Recipe_generator\venv\lib\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\badje\OneDrive\Bureau\Cours_M2\NLP\projet\Recipe_generator\venv\lib\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\badje\OneDrive\Bureau\Cours_M2\NLP\projet\Recipe_generator\venv\lib\site-packages\transformers\modeling_tf_utils.py", line 1476, in train_step
        data = data_adapter.expand_1d(data)

    AttributeError: module 'keras.engine.data_adapter' has no attribute 'expand_1d'
